In [132]:
# TensorFlow and tf.keras
import math
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
from tensorflow import keras

from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions
import tensorflow_hub as hub

from tensorboard.plugins.hparams import api as hp
import matplotlib.pyplot as plt


#tf version should be 2.5 or higher
tf.__version__

'2.6.0'

In [ ]:
(raw_train, raw_val, raw_test), metadata=tfds.load('oxford_flowers102', split=['train', 'test', 'validation'], with_info=True, as_supervised=True)


In [ ]:
get_label_name=metadata.features['label'].int2str
print(metadata.features['label'].int2str)
print(metadata.features['label'].num_classes)

<bound method ClassLabel.int2str of ClassLabel(shape=(), dtype=tf.int64, num_classes=102)>
102


In [ ]:
IMG_SIZE=224

def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image=tf.image.resize(image,(IMG_SIZE,IMG_SIZE))/255.0
  return image, label


In [ ]:
train=raw_train.map(format_example)
test=raw_test.map(format_example)
val=raw_val.map(format_example)

In [ ]:
BATCH_SIZE=32
SHUFFLE_BUFER_SIZE=1000

train_batches=train.shuffle(SHUFFLE_BUFER_SIZE).batch(BATCH_SIZE)
val_batches=val.shuffle(SHUFFLE_BUFER_SIZE).batch(BATCH_SIZE)
test_batches=test.shuffle(SHUFFLE_BUFER_SIZE).batch(BATCH_SIZE)


In [ ]:
IMAGE_SHAPE=(IMG_SIZE, IMG_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE,
                                              include_top=False,
                                              weights='imagenet')

In [ ]:
base_model.trainable = False

In [ ]:


model = tf.keras.Sequential([
  base_model,
  keras.layers.Flatten(),
  keras.layers.Dense(102, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
flatten_3 (Flatten)          (None, 62720)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 102)               6397542   
Total params: 8,655,526
Trainable params: 6,397,542
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:

model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])
  

In [ ]:
def lr_decay(epoch):
    return 0.01*math.pow(0.6, epoch)

lr_decay_callback=tf.keras.callbacks.LearningRateScheduler(lr_decay)

In [ ]:
#train model
history=model.fit(train_batches, epochs=10, validation_data=val_batches, callbacks=[lr_decay_callback])

Epoch 1/10
32/32 [==============================] - 23s 567ms/step - loss: 162.4497 - accuracy: 0.2745 - val_loss: 89.7692 - val_accuracy: 0.4502
Epoch 2/10
32/32 [==============================] - 18s 528ms/step - loss: 11.8333 - accuracy: 0.8941 - val_loss: 53.2977 - val_accuracy: 0.5881
Epoch 3/10
32/32 [==============================] - 18s 531ms/step - loss: 0.8600 - accuracy: 0.9873 - val_loss: 42.8932 - val_accuracy: 0.6430
Epoch 4/10
32/32 [==============================] - 18s 535ms/step - loss: 0.0941 - accuracy: 0.9931 - val_loss: 41.1153 - val_accuracy: 0.6549
Epoch 5/10
32/32 [==============================] - 18s 528ms/step - loss: 6.9304e-08 - accuracy: 1.0000 - val_loss: 41.0484 - val_accuracy: 0.6551
Epoch 6/10
32/32 [==============================] - 18s 531ms/step - loss: 3.1672e-08 - accuracy: 1.0000 - val_loss: 41.0474 - val_accuracy: 0.6551
Epoch 7/10
32/32 [==============================] - 18s 531ms/step - loss: 2.8867e-08 - accuracy: 1.0000 - val_loss: 41.0474 

In [ ]:
accuracy = model.evaluate(test_batches)

32/32 [==============================] - 3s 33ms/step - loss: 37.9701 - accuracy: 0.6686


In [ ]:
base_model_2 = tf.keras.applications.VGG16(input_shape=IMAGE_SHAPE,
                                              include_top=False,
                                              weights='imagenet')

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
base_model_2.trainable = False

In [331]:
model_2 = tf.keras.Sequential([
  base_model_2,

  keras.layers.GlobalAveragePooling2D(),

  keras.layers.Flatten(),
  
  #keras.layers.Dense(1024, use_bias=False),
  #keras.layers.BatchNormalization(center=True, scale=False),
  #keras.layers.Activation('relu'),
  #keras.layers.Dropout(0.3),

  keras.layers.Dense(256, use_bias=False),
  keras.layers.BatchNormalization(center=True, scale=False),
  keras.layers.Activation('relu'),
  keras.layers.Dropout(0.3),
    
  #keras.layers.Dense(128, use_bias=False),
  #keras.layers.BatchNormalization(center=True, scale=False),
  #keras.layers.Activation('relu'),
  #keras.layers.Dropout(0.4),
    
  keras.layers.Dense(102, activation='softmax')
])

In [332]:
model_2.summary()

Model: "sequential_59"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_24  (None, 512)               0         
_________________________________________________________________
flatten_58 (Flatten)         (None, 512)               0         
_________________________________________________________________
dense_131 (Dense)            (None, 256)               131072    
_________________________________________________________________
batch_normalization_71 (Batc (None, 256)               768       
_________________________________________________________________
activation_71 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_71 (Dropout)         (None, 256)             

In [333]:
model_2.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])
  

In [334]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [336]:
#train model
history=model_2.fit(train_batches, epochs=10, validation_data=val_batches, callbacks=[lr_decay_callback])

Epoch 1/10


KeyboardInterrupt: ignored

In [337]:
accuracy = model_2.evaluate(test_batches)

32/32 [==============================] - 4s 67ms/step - loss: 1.6588 - accuracy: 0.5941
